### Extraction

In [ ]:
####### RUN WHEN QUERYING NEW DATA #######

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import json

# Your API Key
api_key = "9203abc1267cb56ec6c9f843271ceef8ce4d3c445923782a5b491296a6f07386"


OpenAQ = {
    "City": ["Bayankhongor", "Baruun-Urt", "Ulgii", "Ulaangom", "Khovd", "Uliastai", "Altai", "Tsetserleg", "Bulgan", "Sukhbaatar", "Darkhan", "Mandalgobi", "Dalanzadgad", "Choir", "Choibalsan", "Undurkhaan", "Sainshand", "Ulaanbaatar"],
    "LocationID": [275089, 230997, 230489, 230483, 230481, 230501, 230505, 525536, 230512, 525556, 525544, 230510, 234145, 232310, 230487, 230480, 230642, 72184],
    "Start": [datetime(2022,1,7), datetime(2021,8,5), datetime(2020,11,25), datetime(2020,11,27), datetime(2020,11,27), datetime(2021,7,29), datetime(2020,11,27), datetime(2023,1,26), datetime(2021,7,30), datetime(2023,1,26), datetime(2023,1,26), datetime(2021,7,30), datetime(2020,11,27), datetime(2021,8,16), datetime(2020,11,27), datetime(2020,11,27), datetime(2023,8,2), datetime(2021,1,19)]
}

OpenAQ_df = pd.DataFrame(OpenAQ)

OpenAQ_df

# Define the API URL
api_url = "https://api.openaq.org/v2/measurements"

# Define the headers with the API Key
headers = {
    "Authorization": f"Bearer {api_key}"
}

# Define the parameters for the API request
params = {
    "parameter": ["pm10", "um003", "humidity", "pm1", "um025", "pm25", "um010", "um050", "um100", "temperature", "pressure", "um005"],
    "limit": 10000  # Increase the limit to get more data per request
}

# Define the date range until now
end_date = datetime.now()

# Initialize an empty list to store data
data_list = []

# Iterate through the OpenAQ dictionary
for city, location_id, start_date in zip(OpenAQ["City"], OpenAQ["LocationID"], OpenAQ["Start"]):
    params["location_id"] = location_id

    # Create a new dictionary to store location name, ID, and data
    location_data = {
        "Location": city,
        "Location_ID": location_id,
        "Data": []  # To store measurement data for this location
    }

    # Perform API requests in a loop with larger date ranges
    while start_date < end_date:
        params["date_from"] = start_date.isoformat() + "Z"

        # Increment the date by three months for the next request
        start_date += timedelta(days=60)  # Interval set to 1 month

        # Set the end_date for each request as the minimum of the next 1 months or the current end_date
        params["date_to"] = min((start_date, end_date)).isoformat() + "Z"

        # Send the API request
        response = requests.get(api_url, params=params, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Convert the response to JSON format
            data = response.json()
            measurements = data.get("results", [])
            location_data["Data"].extend(measurements)
        else:
            print(f"Error: Failed to fetch data from the API. Status code: {response.status_code}")
            break

        # Introduce a delay of 5 seconds before the next request
        time.sleep(5)

    # Add the location's data dictionary to the main data list
    data_list.append(location_data)

    # Introduce a delay of 5 seconds before the next request
    time.sleep(15)

# Now the data_list contains dictionaries, each including location name, location ID, and a list of measurements data for that specific location.
# You can access the data like this:
# for location_data in data_list:
#     location = location_data["Location"]
#     location_id = location_data["Location_ID"]
#     measurements = location_data["Data"]
#     # Process measurements data for this location...


In [ ]:
######## DO NOT RUN #######

# Assuming you have your dictionary of dataframes as 'dataframes_dict'
with open('sensor_data.json', 'w') as file:
    json.dump(data_list, file)

In [ ]:
######## DO NOT RUN #######

import pandas as pd
import json

# Load the JSON-serialized dictionary from the file
with open('Data/sensor_data.json', 'r') as file:
    dataframes_dict_json_serializable = json.load(file)

# Convert the JSON-serialized dictionary back to DataFrames
dataframes_dict = {}
for key, data in dataframes_dict_json_serializable.items():
    dataframes_dict[key] = pd.DataFrame(data)

In [ ]:
dataframes_dict["Bayankhongor"]

In [ ]:
######## RUN THIS TO SENSOR FILE ###########

# Create an empty dictionary to store DataFrames for each city
city_dataframes = {}

# Loop through the location_data list
for location_data in data_list:
    city = location_data["Location"]

    # Extract the measurement data for the current city
    measurements = location_data["Data"]

    # Create a DataFrame from the measurement data
    df_city = pd.DataFrame(measurements)

    # Store the DataFrame in the city_dataframes dictionary with the city name as the key
    city_dataframes[city] = df_city

# Now the city_dataframes dictionary contains DataFrames for each city.
df_bayankhongor = city_dataframes["Bayankhongor"]
df_baruun_urt = city_dataframes["Baruun-Urt"]
df_ulgii = city_dataframes["Ulgii"]
df_ulaangom = city_dataframes["Ulaangom"]
df_khovd = city_dataframes["Khovd"] 
df_uliastai = city_dataframes["Uliastai"] 
df_altai = city_dataframes["Altai"]
df_tsetserleg = city_dataframes["Tsetserleg"]
df_bulgan = city_dataframes["Bulgan"]
df_sukhbaatar = city_dataframes["Sukhbaatar"]
df_darkhan = city_dataframes["Darkhan"]
df_mandalgobi = city_dataframes["Mandalgobi"] 
df_dalanzadgad = city_dataframes["Dalanzadgad"]
df_choir = city_dataframes["Choir"]
df_choibalsan = city_dataframes["Choibalsan"]
df_undurkhaan = city_dataframes["Undurkhaan"]
df_sainshand = city_dataframes["Sainshand"]
df_ulaanbaatar = city_dataframes["Ulaanbaatar"]

dataframes_dict_json_serializable = {}
for key, df in city_dataframes.items():
    dataframes_dict_json_serializable[key] = df.to_dict(orient='list')

# Save the dictionary to a JSON file
with open('Data/sensor_data.json', 'w') as file:
    json.dump(dataframes_dict_json_serializable, file)

In [ ]:
df_ulaanbaatar

In [ ]:
city_dataframes = {
    "Bayankhongor": df_bayankhongor,
    "Baruun-Urt": df_baruun_urt,
    "Ulgii": df_ulgii,
    "Ulaangom": df_ulaangom,
    "Khovd": df_khovd,
    "Uliastai": df_uliastai,
    "Altai": df_altai,
    "Tsetserleg": df_tsetserleg,
    "Bulgan": df_bulgan,
    "Sukhbaatar": df_sukhbaatar,
    "Darkhan": df_darkhan,
    "Mandalgobi": df_mandalgobi,
    "Dalanzadgad": df_dalanzadgad,
    "Choir": df_choir,
    "Choibalsan": df_choibalsan,
    "Undurkhaan": df_undurkhaan,
    "Sainshand": df_sainshand,
    "Ulaanbaatar": df_ulaanbaatar
}

# Function to print the shape of each DataFrame
def print_dataframe_shapes(city_dataframes):
    for city, df in city_dataframes.items():
        print(f"City: {city}, Shape: {df.shape}")

# Call the function to print the shapes of each DataFrame
print_dataframe_shapes(city_dataframes)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Define the API URL
api_url = "https://api.openaq.org/v2/measurements"

# Define the parameters for the API request
params = {
    "location_id": 234145,
    "parameter": ["pm10", "um003", "humidity", "pm1", "um025", "pm25", "um010", "um050", "um100", "temperature", "pressure", "um005"],
    "limit": 50000  # Increase the limit to get more data per request
}

# Define the date range from January 1, 2020, to now
start_date = datetime(2020, 11, 27)
end_date = datetime.now()

# Initialize an empty list to store data
data_list = []

# Perform API requests in a loop with larger date ranges
while start_date < end_date:
    params["date_from"] = start_date.isoformat() + "Z"

    # Increment the date by three months for the next request
    start_date += timedelta(days=90)  # Increase interval to 3 months

    # Set the end_date for each request as the minimum of the next 3 months or the current end_date
    params["date_to"] = min((start_date, end_date)).isoformat() + "Z"

    # Send the API request
    response = requests.get(api_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()
        measurements = data.get("results", [])
        data_list.extend(measurements)
    else:
        print(f"Error: Failed to fetch data from the API. Status code: {response.status_code}")
        break

    # Introduce a delay of 5 seconds before the next request
    time.sleep(15)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

In [ ]:
df

In [ ]:
OpenAQ = {
    "City": ["Bayankhongor", "Baruun-Urt", "Ulgii", "Ulaangom", "Khovd", "Uliastai", "Altai", "Tsetserleg", "Bulgan", "Sukhbaatar", "Darkhan", "Mandalgobi", "Dalanzadgad", "Choir", "Choibalsan", "Undurkhaan", "Sainshand", "Ulaanbaatar"],
    "LocationID": [275089, 230997, 230489, 230483, 230481, 230501, 230505, 525536, 230512, 525556, 525544, 230510, 234145, 232310, 230487, 230480, 230642, 72184],
    "Start": [datetime(2022,1,7), datetime(2021,8,5), datetime(2020,11,25), datetime(2020,11,27), datetime(2020,11,27), datetime(2021,7,29), datetime(2020,11,27), datetime(2023,1,26), datetime(2021,7,30), datetime(2023,1,26), datetime(2023,1,26), datetime(2021,7,30), datetime(2020,11,27), datetime(2021,8,16), datetime(2020,11,27), datetime(2020,11,27), datetime(2023,8,2), datetime(2021,1,19)]
}

OpenAQ_df = pd.DataFrame(OpenAQ)

OpenAQ_df